In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np
import pprint
from datetime import datetime

class game_info_by_match_id():
    """
    Returns df of user info from a given match (so far).
    """
    #define private variables to use in class

    def __init__(self, api_key, name, region, gamemode, gameid):
        #upon calling the class we pass in a bunch of things to initialize^
        self.api_key = api_key
        self.name = name
        self.region = region
        self.gamemode = gamemode
        self.gameid = gameid
        watcher = LolWatcher(self.api_key)
        self.user = watcher.summoner.by_name(region, name)

    def rank_stats(self):
        watcher = LolWatcher(self.api_key)

        #league, division, games played, etc.
        encrypted_summoner_id = self.user['id']
        try:
            self.rank_stats = watcher.league.by_summoner(self.region, self.user['id'])
        except:
            self.rank_stats = {'tier': ['unranked']}
        return self.rank_stats
    
    def match_data(self):
        watcher = LolWatcher(self.api_key)

        self.matches = watcher.match.matchlist_by_account(self.region, self.user['accountId'])

        self.match_data = watcher.match.by_id(self.region, self.gameid)
        m = self.match_data

        #n is for each "participant" or player in the match
        def gd():
            n = [] #dump raw stats into here from participants
            for row in m['participants']:
                m_row = {}
                m_row['champion'] = row['championId']
                m_row['spell1'] = row['spell1Id']
                m_row['spell2'] = row['spell2Id']
                m_row['teamId'] = row['teamId']
                win_lose = row['stats']['win']
                if win_lose == True:
                    win_lose = '胜利'
                else:
                    win_lose = '失败'
                m_row['win'] = win_lose

                m_row['kills'] = row['stats']['kills']
                m_row['deaths'] = row['stats']['deaths']
                m_row['assists'] = row['stats']['assists']
                m_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
                m_row['totalDamageDealtToChampions'] = row['stats']['totalDamageDealtToChampions']
                m_row['goldEarned'] = row['stats']['goldEarned']
                m_row['champLevel'] = row['stats']['champLevel']
                m_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
                m_row['item0'] = row['stats']['item0']
                m_row['item1'] = row['stats']['item1']
                m_row['item2'] = row['stats']['item2']
                m_row['item3'] = row['stats']['item3']
                m_row['item4'] = row['stats']['item4']
                m_row['item5'] = row['stats']['item5']
                m_row['item6'] = row['stats']['item6']
                m_row['firstBloodKill'] = row['stats']['firstBloodKill']
                m_row['firstBloodAssist'] = row['stats']['firstBloodAssist']
                m_row['visionWardsBoughtInGame'] = row['stats']['visionWardsBoughtInGame']
                m_row['visionScore'] = row['stats']['visionScore']
                m_row['creepsPerMinDeltas'] = row['timeline']['creepsPerMinDeltas']
                m_row['goldPerMinDeltas'] = row['timeline']['goldPerMinDeltas']
                m_row['lane'] = row['timeline']['lane']
                m_row['ccScore'] = row['stats']['totalTimeCrowdControlDealt']
                m_row['perkPrimaryStyle'] = row['stats']['perkPrimaryStyle']
                m_row['perkSubStyle'] = row['stats']['perkSubStyle']
                n.append(m_row)
            return n

        n = gd()
        for i in range(0,len(n)):
            n[i]['summonerName'] = m['participantIdentities'][i]['player']['summonerName']
            n[i]['profileIcon'] = m['participantIdentities'][i]['player']['profileIcon']

        latest = watcher.data_dragon.versions_for_region(self.region)['n']['champion']
        static_champ_list = watcher.data_dragon.champions(latest, False, 'en_US')
        static_item_list = watcher.data_dragon.items(latest, 'en_US')
        static_summonerspell_list = watcher.data_dragon.summoner_spells(latest, 'en_US')

        def g_c(n): #gets summoner spells, champions, and items

            #summoner spells
            spell_url = "http://ddragon.leagueoflegends.com/cdn/11.2.1/img/spell/"

            summonerspell_dict = {}
            for key in static_summonerspell_list['data']:
                row = static_summonerspell_list['data'][key]
                summonerspell_dict[row['key']] = row['id']
                summonerspell_dict[row['key']] = spell_url + str(row['image']['full'])

            #champs
            champ_url = "https://ddragon.leagueoflegends.com/cdn/11.2.1/img/champion/"

            champ_name_dict = {}
            champ_image_dict = {}
            for key in static_champ_list['data']:
                row = static_champ_list['data'][key]
                champ_name_dict[row['key']] = row['id']
               # champ_dict[row['image']] = champ_url + str(row['image']['full'])
                champ_image_dict[row['key']] = champ_url +  str(row['image']['full'])

            #items
            item_url = "https://ddragon.leagueoflegends.com/cdn/11.2.1/img/item/"

            item_dict = {}

            for key in static_item_list['data']:
                row = static_item_list['data'][key]
                item_dict[key] = row['name']
            #add to df
            for row in n:
                #print(str(row['item1']) + ' ' + item_dict[str(row['item1'])])
                row['championName'] = champ_name_dict[str(row['champion'])]
                row['championImage'] = champ_image_dict[str(row['champion'])]
                for i in range(0,7):
                    try:
                        row['itemName' +str(i)] = item_dict[str(row['item'+str(i)])]
                        row['itemImage' + str(i)] = item_url + str(row['item'+str(i)]) + '.png'
                    except:
                        row['itemName' +str(i)] = 0
                row['spell1Image'] = summonerspell_dict[str(row['spell1'])]
                row['spell2Image'] = summonerspell_dict[str(row['spell2'])]

                row['profileIconImage'] = 'http://ddragon.leagueoflegends.com/cdn/11.2.1/img/profileicon/' + str(row['profileIcon']) + '.png'

            df = pd.DataFrame(n)
            return df



        df = g_c(n)
        #add in extra columns
        df['gameDuration'] = round((m['gameDuration'] / 60),2)
        df['gameMode'] = m['gameMode']
        df['gameCreation'] = m['gameCreation'] / 1000
        df['kda'] = ((df['kills'] + df['assists']) / df['deaths']).round(2)
        df['killParticipation'] = ((df['kills'] + df['assists'])/ df.groupby('teamId')['kills'].transform(np.sum) * 100).astype(int)
        df['minionsKilledPerMinute'] = (df['totalMinionsKilled'] / df['gameDuration']).round(1)

        #get time since last played (in days)
        now = datetime.now()
        last_game_played_when = df['gameCreation'].values[0]
        last_game_played_when = datetime.fromtimestamp(last_game_played_when)

        df['lastGamePlayedWhen'] = (now - last_game_played_when).days

        #get first blood, baron kills, etc.
        def g_t(df, m):
            def x(y):
                m_id = m['teams'][0]['teamId']
                m_team = m['teams'][0]
                return np.where(df['teamId']==m_id, m_team[y], m['teams'][0+1][y])
            l = ['firstBlood', 'baronKills', 'firstTower', 'firstRiftHerald', 'towerKills',
                 'inhibitorKills', 'dragonKills','riftHeraldKills']
            for i in l:
                df[i] = x(i)
            return df

        df = g_t(df, m)

        return df


In [8]:
api_key = 'RGAPI-573a56a6-8a7e-4de4-b257-c3437db4a91d'
region = 'na1'
gamemode = 'CLASSIC'
name = 'Sir Thiccboi'

watcher = LolWatcher(api_key)
user = watcher.summoner.by_name('na1', name)
matches = watcher.match.matchlist_by_account(region, user['accountId'])
game_ids = []
for i in range(0,2): #display 10 games
    game_ids.append(matches['matches'][i]['gameId'])

# onload_display = pd.DataFrame()
# for game in game_ids:
#     gameid = game
gameid = game_ids[0]
df = game_info_by_match_id(api_key,
                                      name, region,
                                      gamemode, gameid).match_data()

df2 =  game_info_by_match_id(api_key,
                                      name, region,
                                      gamemode, gameid).rank_stats()


In [9]:
df2 = pd.DataFrame(df2)

In [4]:
if df2.empty:
    df2 = pd.DataFrame({'tier': ['unranked']})

In [5]:
if df2['tier'].values[0] == 'unranked':
    print('asdlkj')

asdlkj


In [6]:
pd.set_option('display.max_columns', None)
df

,champion,spell1,spell2,teamId,win,kills,deaths,assists,totalDamageDealt,totalDamageDealtToChampions,goldEarned,champLevel,totalMinionsKilled,item0,item1,item2,item3,item4,item5,item6,firstBloodKill,firstBloodAssist,visionWardsBoughtInGame,visionScore,creepsPerMinDeltas,goldPerMinDeltas,lane,ccScore,perkPrimaryStyle,perkSubStyle,summonerName,profileIcon,championName,championImage,itemName0,itemImage0,itemName1,itemImage1,itemName2,itemImage2,itemName3,itemImage3,itemName4,itemImage4,itemName5,itemImage5,itemName6,itemImage6,spell1Image,spell2Image,profileIconImage,gameDuration,gameMode,gameCreation,kda,killParticipation,minionsKilledPerMinute,lastGamePlayedWhen,firstBlood,baronKills,firstTower,firstRiftHerald,towerKills,inhibitorKills,dragonKills,riftHeraldKills
0,81,4,11,100,失败,8,6,19,142824,42702,15603,18,31,3042,3158,6609,3153,3078,1036,3340,False,False,0,3,{'0-10': 0.8999999999999999},{'0-10': 521.8},NONE,3,8000,8200,ye qiu11,4832,Ezreal,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Muramana,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Ionian Boots of Lucidity,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Chempunk Chainsword,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Blade of The Ruined King,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Trinity Force,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Long Sword,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Stealth Ward,https://ddragon.leagueoflegends.com/cdn/11.2.1...,http://ddragon.leagueoflegends.com/cdn/11.2.1/...,http://ddragon.leagueoflegends.com/cdn/11.2.1/...,http://ddragon.leagueoflegends.com/cdn/11.2.1/...,19.7,NEXUSBLITZ,1.608451e+09,4.50,84,1.6,44,True,0,False,False,0,0,0,0
1,517,4,14,100,失败,9,9,13,82267,29726,14132,18,37,1058,3157,3020,3165,1058,4636,3340,False,False,0,4,{'0-10': 2.7},{'0-10': 638.9000000000001},NONE,205,8000,8100,SwollenGooch,512,Sylas,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Needlessly Large Rod,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Zhonya's Hourglass,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Sorcerer's Shoes,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Morellonomicon,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Needlessly Large Rod,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Night Harvester,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Stealth Ward,https://ddragon.leagueoflegends.com/cdn/11.2.1...,http://ddragon.leagueoflegends.com/cdn/11.2.1/...,http://ddragon.leagueoflegends.com/cdn/11.2.1/...,http://ddragon.leagueoflegends.com/cdn/11.2.1/...,19.7,NEXUSBLITZ,1.608451e+09,2.44,68,1.9,44,True,0,False,False,0,0,0,0
2,154,4,11,100,失败,2,8,22,113567,15654,14343,18,30,3143,3076,3068,3111,3065,3066,3364,False,False,0,5,{'0-10': 0.4},{'0-10': 528.5},NONE,308,8000,8400,Sir Thiccboi,4855,Zac,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Randuin's Omen,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Bramble Vest,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Sunfire Aegis,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Mercury's Treads,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Spirit Visage,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Winged Moonplate,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Oracle Lens,https://ddragon.leagueoflegends.com/cdn/11.2.1...,http://ddragon.leagueoflegends.com/cdn/11.2.1/...,http://ddragon.leagueoflegends.com/cdn/11.2.1/...,http://ddragon.leagueoflegends.com/cdn/11.2.1/...,19.7,NEXUSBLITZ,1.608451e+09,3.00,75,1.5,44,True,0,False,False,0,0,0,0
3,34,7,4,100,失败,7,7,16,203855,28737,18028,18,111,6653,3157,3020,3165,4637,3040,3340,False,False,0,2,{'0-10': 4},{'0-10': 601},NONE,1407,8100,8000,Divine Right,582,Anivia,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Liandry's Anguish,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Zhonya's Hourglass,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Sorcerer's Shoes,https://ddragon.leagueoflegends.com/cdn/11.2.1...,Morellonomicon,https://ddragon.leagueoflegends.com/cdn/11.2.

In [40]:
df['gameDuration']

0    27.05
1    27.05
2    27.05
3    27.05
4    27.05
5    27.05
6    27.05
7    27.05
8    27.05
9    27.05
Name: gameDuration, dtype: float64

In [54]:
 def game_dur(x):
    x = df['gameDuration'][0]
    x = str(x).split('.')
    return x[0] + ' min ' + x[1] + ' sec'

In [56]:
df['gameDuration'] = game_dur(df['gameDuration'].values[0])

In [57]:
df['gameDuration']

0    27 min 05 sec
1    27 min 05 sec
2    27 min 05 sec
3    27 min 05 sec
4    27 min 05 sec
5    27 min 05 sec
6    27 min 05 sec
7    27 min 05 sec
8    27 min 05 sec
9    27 min 05 sec
Name: gameDuration, dtype: object

In [10]:
df2

,leagueId,queueType,tier,rank,summonerId,summonerName,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,1c962c51-250e-4269-ba1a-beb7bf7212e0,RANKED_SOLO_5x5,GOLD,III,f4S_T9rOJ5Axs4elcugsh0T6Se0Mhi2gj9i-kv_bXLmmfxc,Sir Thiccboi,6,45,45,False,False,False,True


In [46]:
df['gameDuration'] = df['gameDuration'].values.split('.')

AttributeError: 'numpy.ndarray' object has no attribute 'split'